# 🚀 Day 2: Warp Primitives & Shuffle Operations

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdodlapati3/cuda-lab/blob/main/learning-path/week-04/day-2-warp-primitives.ipynb)

## Learning Objectives
- Understand warp execution model (32 threads in lockstep)
- Use warp shuffle operations (__shfl_down, __shfl_xor)
- Implement warp-level reduction without shared memory
- Combine warp and block reduction for optimal performance

> **Primary Focus:** CUDA C++ code examples first, Python/Numba backup for interactive testing

---

In [ ]:
# ⚙️ Colab/Local Setup - Run this first!
import subprocess, sys
try:
    import google.colab
    print("🔧 Running on Google Colab - Installing dependencies...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
    print("✅ Setup complete!")
except ImportError:
    print("💻 Running locally - make sure you have: pip install numba numpy")

import numpy as np
from numba import cuda
import math
import time

print(f"\nCUDA available: {cuda.is_available()}")
if cuda.is_available():
    device = cuda.get_current_device()
    print(f"Device: {device.name}")
    print(f"Warp size: {device.WARP_SIZE}")

---

## Part 1: Understanding Warps

### What is a Warp?

```
Block of 256 threads:
┌─────────────────────────────────────────────────────────────────────┐
│ Warp 0     │ Warp 1     │ Warp 2     │ ... │ Warp 7     │
│ tid 0-31   │ tid 32-63  │ tid 64-95  │ ... │ tid 224-255│
└─────────────────────────────────────────────────────────────────────┘

• A warp = 32 threads that execute in LOCKSTEP
• All 32 threads execute the same instruction at the same time
• Threads in a warp can directly exchange data (no shared memory!)
```

### CUDA C++ Warp Shuffle Operations (Primary)

The following CUDA C++ implementation demonstrates warp-level shuffle operations for efficient reduction without shared memory.

In [ ]:
%%writefile warp_primitives.cu
// warp_primitives.cu - Warp shuffle operations
#include <stdio.h>
#include <cuda_runtime.h>

// Warp-level reduction using shuffle
__device__ float warpReduceSum(float val) {
    // Use __shfl_down_sync to reduce within warp
    // All threads in warp must participate (mask = 0xffffffff)
    for (int offset = 16; offset > 0; offset >>= 1) {
        val += __shfl_down_sync(0xffffffff, val, offset);
    }
    return val;  // Only lane 0 has the final sum
}

__device__ float warpReduceMax(float val) {
    for (int offset = 16; offset > 0; offset >>= 1) {
        float other = __shfl_down_sync(0xffffffff, val, offset);
        val = fmaxf(val, other);
    }
    return val;
}

// Block-level reduction using warps
__global__ void blockReduceSumWarp(const float* input, float* output, int n) {
    // Each thread loads and accumulates via grid-stride
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    
    float sum = 0.0f;
    for (int i = gid; i < n; i += stride) {
        sum += input[i];
    }
    
    // Warp-level reduction
    sum = warpReduceSum(sum);
    
    // First thread in each warp writes to shared memory
    __shared__ float warpSums[32];  // Max 32 warps per block
    int warpId = tid / 32;
    int laneId = tid % 32;
    
    if (laneId == 0) {
        warpSums[warpId] = sum;
    }
    __syncthreads();
    
    // First warp reduces all warp sums
    if (warpId == 0) {
        sum = (tid < blockDim.x / 32) ? warpSums[tid] : 0.0f;
        sum = warpReduceSum(sum);
        
        if (tid == 0) {
            output[blockIdx.x] = sum;
        }
    }
}

// Other useful warp operations:
// __shfl_sync(mask, val, srcLane)    - Get value from srcLane
// __shfl_up_sync(mask, val, delta)   - Get value from (laneId - delta)
// __shfl_down_sync(mask, val, delta) - Get value from (laneId + delta)
// __shfl_xor_sync(mask, val, mask)   - Get value from (laneId XOR mask)

int main() {
    int n = 1000000;
    size_t size = n * sizeof(float);
    
    // Allocate and initialize
    float *h_input = (float*)malloc(size);
    for (int i = 0; i < n; i++) {
        h_input[i] = 1.0f;
    }
    
    float *d_input, *d_output;
    cudaMalloc(&d_input, size);
    cudaMalloc(&d_output, 256 * sizeof(float));
    
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);
    
    blockReduceSumWarp<<<256, 256>>>(d_input, d_output, n);
    cudaDeviceSynchronize();
    
    // Get partial results and sum on CPU
    float h_output[256];
    cudaMemcpy(h_output, d_output, 256 * sizeof(float), cudaMemcpyDeviceToHost);
    
    float total = 0.0f;
    for (int i = 0; i < 256; i++) {
        total += h_output[i];
    }
    
    printf("Warp reduction sum of %d elements: %f (expected %d)\n", n, total, n);
    
    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o warp_primitives warp_primitives.cu
!./warp_primitives

In [ ]:
# Python equivalent for understanding warp structure
@cuda.jit
def show_warp_info(output, n):
    """Display warp information for each thread."""
    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    
    if gid < n:
        # Calculate warp ID and lane ID
        warp_id = tid // 32  # Which warp within the block
        lane_id = tid % 32   # Position within the warp (0-31)
        
        # Pack info: warp_id * 100 + lane_id
        output[gid] = warp_id * 100 + lane_id

# Test
n = 64
d_output = cuda.device_array(n, dtype=np.int32)

show_warp_info[1, 64](d_output, n)
output = d_output.copy_to_host()

print("Thread → Warp.Lane")
print("="*40)
for i in range(n):
    warp = output[i] // 100
    lane = output[i] % 100
    if i % 8 == 0:
        print()
    print(f"T{i:2d}→W{warp}.L{lane:2d}", end="  ")
print()

---

## Part 2: Warp Shuffle Operations

### The Magic of Shuffle

```
Shuffle allows threads in a warp to read each other's registers!
No shared memory needed. No synchronization needed.

shuffle_sync(mask, val, src_lane):
  - mask: which threads participate (0xFFFFFFFF = all 32)
  - val: the value to share
  - src_lane: which lane to read from

Example: If lane 5 has val=42, then shuffle_sync(mask, val, 5)
         returns 42 for ALL threads in the warp!
```

In [ ]:
# Note: Numba's warp shuffle support
# cuda.shfl_sync(mask, value, src_lane)
# cuda.shfl_down_sync(mask, value, delta)
# cuda.shfl_up_sync(mask, value, delta)
# cuda.shfl_xor_sync(mask, value, lane_mask)

@cuda.jit
def demo_shfl_sync(output, n):
    """Demo: Read value from lane 0."""
    tid = cuda.threadIdx.x
    
    if tid < n:
        # Each thread has its own value
        my_val = tid * 10
        
        # All threads read from lane 0
        # 0xFFFFFFFF = all 32 threads participate
        val_from_lane0 = cuda.shfl_sync(0xFFFFFFFF, my_val, 0)
        
        output[tid] = val_from_lane0

d_out = cuda.device_array(32, dtype=np.int32)
demo_shfl_sync[1, 32](d_out, 32)
result = d_out.copy_to_host()

print("shfl_sync: All threads read lane 0's value (0)")
print(f"Result: {result[:8]}... (all should be 0)")

In [ ]:
@cuda.jit
def demo_shfl_down(output, n):
    """Demo: shfl_down shifts values down by delta lanes."""
    tid = cuda.threadIdx.x
    
    if tid < n:
        my_val = tid  # Lane i has value i
        
        # Each thread reads from lane (tid + 1)
        # Lane 0 gets value from lane 1
        # Lane 1 gets value from lane 2
        # etc.
        val = cuda.shfl_down_sync(0xFFFFFFFF, my_val, 1)
        
        output[tid] = val

d_out = cuda.device_array(32, dtype=np.int32)
demo_shfl_down[1, 32](d_out, 32)
result = d_out.copy_to_host()

print("shfl_down_sync(val, 1): Each thread reads from lane+1")
print(f"Lane 0 gets {result[0]} (from lane 1)")
print(f"Lane 1 gets {result[1]} (from lane 2)")
print(f"Lane 30 gets {result[30]} (from lane 31)")
print(f"Lane 31 gets {result[31]} (undefined, wraps)")

---

## Part 3: Warp-Level Reduction

### Using shfl_down for Sum

In [ ]:
@cuda.jit(device=True)
def warp_reduce_sum(val):
    """
    Reduce 32 values (one per lane) to a single sum.
    Result is in lane 0.
    
    This uses the "butterfly" reduction pattern:
    
    Initial: [0] [1] [2] [3] [4] [5] [6] [7] ... [31]
    
    Step 1 (delta=16): lanes 0-15 add lanes 16-31
    Step 2 (delta=8):  lanes 0-7 add lanes 8-15
    Step 3 (delta=4):  lanes 0-3 add lanes 4-7
    Step 4 (delta=2):  lanes 0-1 add lanes 2-3
    Step 5 (delta=1):  lane 0 adds lane 1
    
    Final: lane 0 has the sum of all 32 values!
    """
    # Full warp participates
    mask = 0xFFFFFFFF
    
    # Step through decreasing offsets
    val += cuda.shfl_down_sync(mask, val, 16)
    val += cuda.shfl_down_sync(mask, val, 8)
    val += cuda.shfl_down_sync(mask, val, 4)
    val += cuda.shfl_down_sync(mask, val, 2)
    val += cuda.shfl_down_sync(mask, val, 1)
    
    return val

In [ ]:
@cuda.jit
def test_warp_reduce(output):
    """Test warp reduction: sum values 0-31."""
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    # Each lane has its lane number as value
    val = float(lane)
    
    # Reduce within warp
    sum_val = warp_reduce_sum(val)
    
    # Lane 0 writes result
    if lane == 0:
        output[0] = sum_val

d_out = cuda.device_array(1, dtype=np.float32)
test_warp_reduce[1, 32](d_out)
result = d_out.copy_to_host()[0]

expected = sum(range(32))  # 0+1+2+...+31 = 496
print(f"Warp reduce sum of lanes 0-31: {result:.0f}")
print(f"Expected (0+1+...+31): {expected}")
print(f"Match: {'✓' if result == expected else '✗'}")

In [ ]:
# Visualize the butterfly pattern
def visualize_warp_reduction():
    print("Warp Reduction (shfl_down pattern)")
    print("="*60)
    print()
    
    # Simplified to 8 lanes for visualization
    vals = list(range(8))
    print(f"Initial: {vals}")
    print()
    
    for delta in [4, 2, 1]:
        new_vals = vals.copy()
        for i in range(8 - delta):
            new_vals[i] = vals[i] + vals[i + delta]
        print(f"After delta={delta}:")
        print(f"  Lane 0 = {vals[0]} + {vals[delta]} = {new_vals[0]}")
        vals = new_vals
        print(f"  Values: {vals}")
        print()
    
    print(f"Final sum in lane 0: {vals[0]}")
    print(f"Expected: {sum(range(8))}")

visualize_warp_reduction()

---

## Part 4: Combining Warp + Block Reduction

### Optimal Block Reduction Pattern

In [ ]:
@cuda.jit
def block_reduce_warp_optimized(arr, block_results, n):
    """
    Optimized block reduction using warp primitives.
    
    Strategy:
    1. Each thread accumulates its portion (grid-stride)
    2. Warp-level reduction (no shared memory, no sync!)
    3. Lane 0 of each warp writes to shared memory
    4. Final warp reduces the warp sums
    """
    # Shared memory for warp results (max 32 warps per block)
    warp_sums = cuda.shared.array(32, dtype=np.float32)
    
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    gid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    lane = tid % 32
    warp_id = tid // 32
    num_warps = cuda.blockDim.x // 32
    
    # Phase 1: Grid-stride accumulation
    local_sum = 0.0
    for i in range(gid, n, stride):
        local_sum += arr[i]
    
    # Phase 2: Warp-level reduction
    warp_sum = warp_reduce_sum(local_sum)
    
    # Phase 3: Lane 0 of each warp stores result
    if lane == 0:
        warp_sums[warp_id] = warp_sum
    
    cuda.syncthreads()
    
    # Phase 4: First warp reduces all warp sums
    if warp_id == 0:
        # Load warp sum (or 0 if beyond num_warps)
        val = warp_sums[lane] if lane < num_warps else 0.0
        
        # Final warp reduction
        final_sum = warp_reduce_sum(val)
        
        if lane == 0:
            block_results[bid] = final_sum

In [ ]:
def gpu_reduce_sum_warp(arr):
    """GPU sum using warp-optimized reduction."""
    n = len(arr)
    blocks = 256
    threads = 256
    
    d_arr = cuda.to_device(arr)
    d_block_results = cuda.device_array(blocks, dtype=np.float32)
    
    # First pass
    block_reduce_warp_optimized[blocks, threads](d_arr, d_block_results, n)
    
    # Second pass
    d_final = cuda.device_array(1, dtype=np.float32)
    block_reduce_warp_optimized[1, threads](d_block_results, d_final, blocks)
    
    return d_final.copy_to_host()[0]

In [ ]:
# Test correctness
n = 10_000_000
arr = np.random.rand(n).astype(np.float32)

gpu_result = gpu_reduce_sum_warp(arr)
cpu_result = np.sum(arr)

print(f"Warp-optimized reduction test")
print(f"GPU: {gpu_result:.2f}")
print(f"CPU: {cpu_result:.2f}")
print(f"Match: {'✓' if np.isclose(gpu_result, cpu_result, rtol=1e-4) else '✗'}")

---

## Part 5: Other Warp Operations

### Warp Max and Min

In [ ]:
@cuda.jit(device=True)
def warp_reduce_max(val):
    """Reduce to maximum value across warp."""
    mask = 0xFFFFFFFF
    
    other = cuda.shfl_down_sync(mask, val, 16)
    val = max(val, other)
    other = cuda.shfl_down_sync(mask, val, 8)
    val = max(val, other)
    other = cuda.shfl_down_sync(mask, val, 4)
    val = max(val, other)
    other = cuda.shfl_down_sync(mask, val, 2)
    val = max(val, other)
    other = cuda.shfl_down_sync(mask, val, 1)
    val = max(val, other)
    
    return val

@cuda.jit(device=True)
def warp_reduce_min(val):
    """Reduce to minimum value across warp."""
    mask = 0xFFFFFFFF
    
    other = cuda.shfl_down_sync(mask, val, 16)
    val = min(val, other)
    other = cuda.shfl_down_sync(mask, val, 8)
    val = min(val, other)
    other = cuda.shfl_down_sync(mask, val, 4)
    val = min(val, other)
    other = cuda.shfl_down_sync(mask, val, 2)
    val = min(val, other)
    other = cuda.shfl_down_sync(mask, val, 1)
    val = min(val, other)
    
    return val

In [ ]:
@cuda.jit
def test_warp_minmax(arr, out_max, out_min):
    """Test warp min/max on first 32 elements."""
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    val = arr[lane]
    
    max_val = warp_reduce_max(val)
    min_val = warp_reduce_min(val)
    
    if lane == 0:
        out_max[0] = max_val
        out_min[0] = min_val

# Test
arr = np.random.rand(32).astype(np.float32)
d_arr = cuda.to_device(arr)
d_max = cuda.device_array(1, dtype=np.float32)
d_min = cuda.device_array(1, dtype=np.float32)

test_warp_minmax[1, 32](d_arr, d_max, d_min)

print(f"Array: {arr[:8]}... (first 8 of 32)")
print(f"GPU max: {d_max.copy_to_host()[0]:.4f}, CPU max: {np.max(arr):.4f}")
print(f"GPU min: {d_min.copy_to_host()[0]:.4f}, CPU min: {np.min(arr):.4f}")

### Warp Broadcast

In [ ]:
@cuda.jit
def demo_warp_broadcast(output):
    """Broadcast a value from one lane to all others."""
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    # Lane 0 has special value
    my_val = 999 if lane == 0 else lane
    
    # Broadcast lane 0's value to all
    broadcast_val = cuda.shfl_sync(0xFFFFFFFF, my_val, 0)
    
    output[tid] = broadcast_val

d_out = cuda.device_array(32, dtype=np.int32)
demo_warp_broadcast[1, 32](d_out)
print(f"Broadcast from lane 0: {d_out.copy_to_host()}")
print("All lanes now have 999!")

---

## Part 6: Performance Comparison

In [ ]:
# Compare shared-memory vs warp-primitive reduction

@cuda.jit
def block_reduce_shared_only(arr, block_results, n):
    """Traditional shared memory reduction."""
    shared = cuda.shared.array(256, dtype=np.float32)
    
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    gid = cuda.grid(1)
    stride = cuda.gridsize(1)
    
    local_sum = 0.0
    for i in range(gid, n, stride):
        local_sum += arr[i]
    
    shared[tid] = local_sum
    cuda.syncthreads()
    
    s = cuda.blockDim.x // 2
    while s > 0:
        if tid < s:
            shared[tid] += shared[tid + s]
        cuda.syncthreads()
        s //= 2
    
    if tid == 0:
        block_results[bid] = shared[0]

In [ ]:
def benchmark_reduction_methods(n, iterations=100):
    """Compare shared-only vs warp-optimized reduction."""
    arr = np.random.rand(n).astype(np.float32)
    d_arr = cuda.to_device(arr)
    
    blocks, threads = 256, 256
    
    # Shared memory only
    d_block1 = cuda.device_array(blocks, dtype=np.float32)
    d_final1 = cuda.device_array(1, dtype=np.float32)
    
    block_reduce_shared_only[blocks, threads](d_arr, d_block1, n)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(iterations):
        block_reduce_shared_only[blocks, threads](d_arr, d_block1, n)
        block_reduce_shared_only[1, threads](d_block1, d_final1, blocks)
    cuda.synchronize()
    shared_time = (time.perf_counter() - start) / iterations * 1000
    
    # Warp-optimized
    d_block2 = cuda.device_array(blocks, dtype=np.float32)
    d_final2 = cuda.device_array(1, dtype=np.float32)
    
    block_reduce_warp_optimized[blocks, threads](d_arr, d_block2, n)
    cuda.synchronize()
    
    start = time.perf_counter()
    for _ in range(iterations):
        block_reduce_warp_optimized[blocks, threads](d_arr, d_block2, n)
        block_reduce_warp_optimized[1, threads](d_block2, d_final2, blocks)
    cuda.synchronize()
    warp_time = (time.perf_counter() - start) / iterations * 1000
    
    return shared_time, warp_time

n = 10_000_000
shared_t, warp_t = benchmark_reduction_methods(n)

print(f"Reduction Benchmark (N={n:,})")
print(f"{'='*40}")
print(f"Shared memory only: {shared_t:.3f} ms")
print(f"Warp-optimized:     {warp_t:.3f} ms")
print(f"Improvement:        {shared_t/warp_t:.2f}x")

---

## Exercises

### Exercise 1: Warp XOR Shuffle

In [ ]:
# TODO: Use shfl_xor_sync to exchange values between paired lanes
# shfl_xor_sync(mask, val, lane_mask) exchanges with lane ^ lane_mask
#
# Example: lane_mask=1 swaps lanes 0↔1, 2↔3, 4↔5, ...
#          lane_mask=2 swaps lanes 0↔2, 1↔3, 4↔6, ...

@cuda.jit
def demo_xor_shuffle(output):
    """Swap values with neighboring lanes using XOR shuffle."""
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    my_val = lane  # Each lane has its ID as value
    
    # TODO: Use cuda.shfl_xor_sync to swap with lane^1
    # swapped = cuda.shfl_xor_sync(0xFFFFFFFF, my_val, 1)
    
    # output[tid] = swapped
    pass

# After XOR with 1: lane 0 should have value 1, lane 1 should have value 0, etc.

### Exercise 2: Warp Prefix Sum (Scan)

In [ ]:
# TODO: Implement inclusive prefix sum within a warp
# Input:  [1, 2, 3, 4, 5, ...]
# Output: [1, 3, 6, 10, 15, ...]
#
# Use shfl_up_sync to get value from lower lane

@cuda.jit(device=True)
def warp_inclusive_scan(val):
    """Inclusive prefix sum within warp."""
    mask = 0xFFFFFFFF
    
    # Step through increasing offsets
    # delta=1: add lane-1
    # delta=2: add lane-2  
    # etc.
    
    # TODO: Implement using shfl_up_sync
    pass

# Test: [1,1,1,...,1] → [1,2,3,...,32]

### Exercise 3: Warp Vote Functions

In [ ]:
# TODO: Explore warp vote functions
# cuda.all_sync(mask, predicate) - True if ALL threads have predicate=True
# cuda.any_sync(mask, predicate) - True if ANY thread has predicate=True
# cuda.ballot_sync(mask, predicate) - Bitmap of which threads have predicate=True

@cuda.jit
def demo_warp_vote(arr, result, n):
    """Check if all/any values in warp are positive."""
    tid = cuda.threadIdx.x
    lane = tid % 32
    
    if lane < n:
        val = arr[lane]
        is_positive = val > 0
        
        # TODO: Use cuda.all_sync and cuda.any_sync
        # all_positive = cuda.all_sync(0xFFFFFFFF, is_positive)
        # any_positive = cuda.any_sync(0xFFFFFFFF, is_positive)
        
        pass

---

## Summary

### Warp Shuffle Operations

| Function | Description | Use Case |
|----------|-------------|----------|
| `shfl_sync` | Read from specific lane | Broadcast |
| `shfl_down_sync` | Read from lane+delta | Reduction |
| `shfl_up_sync` | Read from lane-delta | Prefix sum |
| `shfl_xor_sync` | Read from lane^mask | Butterfly patterns |

### Why Warp Primitives?

```
Shared Memory Reduction:     Warp Shuffle Reduction:
- Write to shared memory     - Direct register exchange
- __syncthreads() needed     - No sync needed in warp!
- Bank conflict possible     - No bank conflicts
- ~8 steps for 256 threads   - 5 steps for 32 lanes
```

### Key Takeaways

1. **Warp = 32 threads in lockstep**
2. **Shuffle = direct register exchange (fast!)**
3. **No synchronization within warp**
4. **Use warp reduction for final 32 values**
5. **Combine: warp reduce → shared memory → warp reduce**

### Next: Day 3 - Atomic Operations
Learn about thread-safe updates for when reduction isn't enough!